**Бинарная классификация** , которая предполагает отнесение объекта к одному из двух классов, часто используется для диагностики заболеваний (например, «болен» или «здоров»), предсказания исходов лечения и других критически важных задач.
**Применение бинарной классификации в медицине:**

* Диагностика заболеваний
Поиск и утверждение заболевания на основе различных показателей. Например, выявление рака по результатам анализа крови или изображений.

* Прогнозирование исходов лечения
Определение вероятности успеха или неудачи определенного курса лечения на основании истории болезни пациента и других данных.

* Скрининг населения
Быстрая и эффективная оценка большого количества пациентов для выявления возможных проблем. Например, скрининг на диабет или гипертонию.

Суть данной задачи заключается в построении разделяющей гиперплоскости в многомерном пространстве признаков. Цель заключается в том, чтобы четко отделить одну область пространства, соответствующую одному классу (например, наличие заболевания), от другой области, соответствующей противоположному классу (например, отсутствие заболевания).

Для успешной реализации бинарной классификации применяется ряд математических и статистических методов, которые позволяют идентифицировать наиболее оптимальную гиперплоскость. Эта гиперплоскость может проходить через пространство в достаточно сложной форме, быть линейной или нелинейной, в зависимости от характеристик и распределения данных. Ключевой задачей является обеспечение точной и надежной классификации новых наблюдений, основывающейся на обучении с использованием существующих данных.

Cначала проведем разведочный анализ данных (Exploratory Data Analysis, EDA) — фундаментальный этап в разработке моделей машинного обучения, включающий в себя всесторонний анализ исходного набора данных, который позволит нам исследовать взаимосвязи между переменными, выявить возможные аномалии и выбросы, а также понять распределение данных и их ключевые параметры.
Итак, будем исследовать набор данных о пациентах heart_data.tsv (1.1.ML_MED_EDA.ipynb), в котором есть целевые признаки — различные сердечно-сосудистые заболевания. Нам необходимо построить модель, которая будет определять, болен человек или нет конкретным заболеванием.

Для начала в качестве целевого признака рассмотрим артериальную гипертензию. Остальные целевые признаки нам пока не нужны.
.tsv в названии датасета означает, что разделителем является знак табуляции (\t). Чтобы таблица загрузилась в привычном виде, необходимо в функции read_csv() библиотеки Pandas указать аргумент sep='\t'.
Датасет содержит числовые и категорийные признаки, есть отсутствующие значения. В этом мы можем убедиться, используя метод heart.info()

Выведем названия всех признаков: heart.columns
Набор данных представляет собой информацию, содержащую различные аспекты демографического, медицинского и поведенческого профиля людей. Он включает в себя следующие признаки:

* идентификационные данные: 'ID';
* демографическая информация: 'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия';
* социально-экономические факторы: 'Образование', 'Профессия', 'Работа', 'Выход на пенсию', 'Прекращение работы по болезни';
* медицинская история: 'Сахарный диабет', 'Гепатит', 'Онкология', 'Хроническое заболевание легких', 'Бронхиальная астма', 'Туберкулез легких', 'ВИЧ/СПИД';
* лекарственная терапия и травмы: 'Регулярный прием лекарственных средств', 'Травмы за год', 'Переломы';
* курение: 'Статус Курения', 'Возраст курения', 'Сигарет в день', 'Пассивное курение', 'Частота пасс кур';
* потребление алкоголя: 'Алкоголь', 'Возраст алког';
* режим сна: 'Время засыпания', 'Время пробуждения', 'Сон после обеда';
* участие в активностях: 'Спорт, клубы', 'Религия, клубы';
* кардиологические состояния: 'Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца'.

Посмотрим статистики числовых признаков с помощью метода .describe(), округлив значения до двух знаков после запятой. Для удобства транспонируем таблицу.

heart.describe().round(2).T

Посмотрим статистики категорийных признаков с помощью того же метода .describe(), используя аргумент include, значением которого укажем 'object':

heart.describe(include='object').T

Можно посмотреть статистики сразу всех переменных, используя значение 'all' для аргумента include:

heart.describe(include='all'). round(2)
Узнаем количество уникальных значений в каждом столбце с помощью метода .nunique().

heart.nunique()

Столбец 'ID' содержит столько же разных значений, сколько элементов (строк) в таблице. Поскольку данная колонка не содержит важной информации о модели, мы можем ее исключить.

Удалим из датасета неинформативный признак 'ID':

heart = heart.drop("ID", axis=1) 
Можно также использовать следующий код:

heart.drop("ID", axis=1, inplace=True)
Здесь inplace=True указывает, что метод drop должен удалить столбец 'ID' из набора данных heart непосредственно, не создавая новую копию данных.

Изучим признак 'ВИЧ/СПИД':

heart['ВИЧ/СПИД'].value_counts(dropna=False)

Всего 2 пациента больны ВИЧ/СПИД, то есть признак является редким. Его также можно отбросить.

Удалим признак 'ВИЧ/СПИД':

heart.drop(['ВИЧ/СПИД'], axis=1, inplace=True)
Отметим, что у людей, которые никогда не курили, отсутствуют значения в столбцах 'Возраст курения' и 'Сигарет в день'.

Значит, мы можем заполнить пропуски, используя максимальное значение возраста для 'Возраст курения'. Например, 90, что будет означать, что пациент начал курить только в 90 лет, точнее никогда не начнет. И ноль для 'Сигарет в день'.

heart['Возраст курения'].fillna(90, inplace=True)
heart['Сигарет в день'].fillna(0, inplace=True)
Аналогично для переменной 'Возраст алког'.

heart['Возраст алког'].fillna(90, inplace=True)

Узнаем, сколько в наших данных пропусков.

Вычислим сумму пропущенных значений в каждом столбце, используя метод isna(), который возвращает таблицу, состоящую из True/False для каждого элемента, в зависимости от того, находится ли там пропущенное значение (NaN) и метод sum(), который суммирует элементы в столбцах таблицы, принимая True за 1, а False за 0.

heart.isna().sum()

Всего в двух признаках есть пропущенные значения: 'Пол' и 'Частота пасс кур'.

У одного человека неизвестен пол. Проанализируем данные этого человека. Выберем только строчку с данными о пациенте, у которого не указан пол:

heart[heart['Пол'].isna()]

Человек, кажется, совершенно здоров, несмотря на то, что курит три сигареты в день (что является необычным количеством для курильщика). Также отсутствуют признаки пассивного курения (указывает на то, что в его окружении курит только один человек — он сам). Возраст начала употребления алкоголя указан 20 лет (больше, чем возраст начала курения — 18 лет). Исходя из времени засыпания и пробуждения, можно предположить, что он спит около пяти с половиной часов, что значительно меньше рекомендуемого количества сна для человека.

В данном случае мы можем закодировать его пол как 0.5, но из-за странных данных лучше удалить пациента из датасета.

Удалим из датасета строчку, в которой не указан пол пациента:

heart.dropna(subset=['Пол'], axis=0, inplace=True)

Женщина курит с 19 лет и указала, что частота пассивного курения 3-6 раз в неделю. Значит, ошибка в переменной 'Пассивное курение'. Заменим значение признака 'Пассивное курение' для пациента с индексом 392 на 1.

Теперь ошибок нет, и мы можем заполнить пропуски в признаке 'Частота пасс кур' нулями.

Заполним пропуски в признаке 'Частота пасс кур'. Так как признак является категорийным, поставим 0 как строку, заключив в кавычки. Иначе при кодировании признака эти нули снова будут рассматриваться как пропущенные значения.
heart['Частота пасс кур'].fillna('0', inplace=True)
Проверим, остались ли у нас пропущенные значения в данных:

heart.isna().sum()
Пропущенных значений нет.


Категориальные переменные требуют соответствующей кодировки для обработки в моделях машинного обучения.

**Существует несколько методов кодирования категориальных признаков:**


* Кодирование порядковых признаков (Ordinal Encoding)
Применяется, когда категориальные признаки имеют явный порядок. Каждой категории присваивается числовое значение на основе ее порядка.

* Кодирование с помощью замены категорий (Label Encoding)
Метод присваивает каждой категории уникальное числовое значение. Кодирование по меткам удобно использовать для категорий, которые не имеют ординала, но может вызывать проблемы в некоторых моделях, таких как линейная регрессия, из-за того, что модель может интерпретировать замены категорий как имеющие порядок.

* Однократное кодирование (One-Hot Encoding)
Этот метод преобразует каждую категорию в новый бинарный признак. Для каждой категории создается отдельный столбец, где у объектов наличия данной категории будет значение 1, а у всех остальных — 0. Это предотвращает проблемы, связанные с порядковыми зависимостями, но увеличивает размерность данных.

* Кодирование среднего значения (Target Encoding)
Этот метод заключается в замене категориальных признаков на средние значения целевой переменной для каждой категории. Метод может быть эффективным, но требует тщательной настройки для предотвращения переобучения, особенно при работе с небольшими датасетами.

* Упорядоченное одноразовое кодирование (Ordered One-Hot Encoding)
В этом методе используется комбинация порядкового кодирования и одноразового кодирования для уменьшения размерности. Категории сначала сортируются, а затем кодируются в стилях «leave-one-out».

* Кодирование бинарных признаков (Binary Encoding)
Этот метод сначала преобразует категории в числа, а затем числа кодируются в двоичный формат. Каждый двоичный разряд кодируется как отдельный признак. Это уменьшает размерность по сравнению с одноразовым кодированием, сохраняя при этом информацию о категории.

* Кодирование количества наблюдений (Frequency Encoding)
Категория заменяется ее частотой или долей, то есть числом ее появления в наборе данных. Это может быть полезным, чтобы захватить информацию о редкости или распространенности категории.

* Кодирование по категориальной эмбеддингу (Category Embedding Encoding)
Метод часто используется в глубоких нейронных сетях. Каждой категории присваивается вектор (эмбеддинг), который обучается вместе с моделью. Это позволяет моделям находить более сложные паттерны и взаимосвязи между категориями.

* Кодирование базовых N получателей (BaseN Encoding)
В этом методе категория сначала преобразуется в ее числовое представление, а затем это число кодируется с использованием меньшей базовой системы счисления (например, в бинарную или тройственную систему). Этот метод позволяет сбалансировать компромисс между размерностью признаков и выраженной информацией.

* Кодирование весов доказательств (Weight of Evidence Encoding)
Каждой категории присваивается значение, связанное с отношением числа положительных и отрицательных примеров для этой категории. Этот метод популярен в кредитном скоринге и других областях, где важна интерпретируемость модели.

* Полиномиальное одноразовое кодирование (Polynomial One-Hot Encoding)
Используется для множественных категорий, где потребуется рассчитать полиномиальные взаимодействия между категориями. Однако это может значительно увеличивать размерность данных и не так часто используется.

**числу наиболее распространенных методов бинарной классификации можно отнести следующие:**

* Логистическая регрессия (Logistic Regression)
Этот метод находит широкое применение благодаря своей простоте и интерпретируемости. Он использует логистическую функцию для моделирования вероятности принадлежности объекта к одной из двух категорий. При этом логистическая регрессия позволяет анализировать влияние нескольких объясняющих переменных и может быть полезна при наличии большого объема данных о пациентах.

* Метод опорных векторов (SVM)
Алгоритм основывается на поиске гиперплоскости, которая максимально разделяет классы в многомерном пространстве. SVM особенно эффективен в случае, когда данные имеют высокую размерность и низкую наблюдаемую выборку, что часто встречается в медицинских исследованиях, таких как анализ генетических данных.

* Деревья решений (Decision Tree)
Метод строит модель в виде дерева, где каждая ветвь представляет собой выбор значения одного из признаков, а листья — соответствующие классы. Деревья решений легко интерпретируются и визуализируются, что делает их полезными для объяснения результатов медицинских исследований. Кроме того, они хорошо справляются с данными, содержащими как числовые, так и категориальные признаки.

* Метод ближайших соседей (kNN)
Этот алгоритм классифицирует объект на основании класса его ближайших соседей в пространстве признаков. Метод kNN является простым и интуитивно понятным, однако его эффективность может значительно изменяться в зависимости от выбора гиперпараметра k и характера распределения данных. В медицине метод может использоваться, например, для классификации пациентов по симптомам или результатам анализов.


Каждый из этих алгоритмов обладает своими сильными и слабыми сторонами. Выбор конкретного метода зависит от специфики задачи, структуры доступных данных и требований к точности и интерпретируемости модели. В контексте медицинских исследований особое внимание также следует уделять проблемам переобучения и выбору адекватных метрик оценки качества классификации.

